In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/optuna-bd-knowledgeextractionnoteb/model.bin
/kaggle/input/optuna-bd-knowledgeextractionnoteb/__results__.html
/kaggle/input/optuna-bd-knowledgeextractionnoteb/__notebook__.ipynb
/kaggle/input/optuna-bd-knowledgeextractionnoteb/__output__.json
/kaggle/input/optuna-bd-knowledgeextractionnoteb/custom.css
/kaggle/input/optuna-bd-knowledgeextractionnoteb/checkpoints/epoch=6-step=28455.ckpt
/kaggle/input/optuna-bd-knowledgeextractionnoteb/checkpoints/epoch=5-step=27316.ckpt
/kaggle/input/optuna-bd-knowledgeextractionnoteb/wandb/run-20210622_142003-30a081bw/run-30a081bw.wandb
/kaggle/input/optuna-bd-knowledgeextractionnoteb/wandb/run-20210622_142003-30a081bw/logs/debug.log
/kaggle/input/optuna-bd-knowledgeextractionnoteb/wandb/run-20210622_142003-30a081bw/logs/debug-internal.log
/kaggle/input/optuna-bd-knowledgeextractionnoteb/wandb/run-20210622_142003-30a081bw/files/wandb-summary.json
/kaggle/input/optuna-bd-knowledgeextractionnoteb/wandb/run-20210622_142003-30a081bw/files/con

In [2]:
!pip install transformers optuna
# !pip install performer-pytorch
!pip install pytorch-lightning wandb -q
# !pip install tkit_mlp_pytorch
os.environ["WANDB_SILENT"] = "true"
os.environ["WANDB_API_KEY"] = "ea464ba3c9768e43f54698ec8b8837c5989d4fcf"

In [3]:
import torch
from transformers import BertTokenizer,AdamW
# from torch.utils.data import 
# from performer_pytorch import PerformerLM
from torch import nn
import torch.nn.functional as F
from torchvision import transforms
# from torchvision.datasets import MNIST
from torch.utils.data import DataLoader, random_split,TensorDataset
import pytorch_lightning as pl
# from tkit_mlp_pytorch import MLP
from transformers import BertTokenizer, BertForMaskedLM,AlbertTokenizer,BertConfig
# import torch\

import optuna
from optuna.integration import PyTorchLightningPruningCallback

# help(DataLoader)
from pytorch_lightning.loggers import WandbLogger
import wandb
# wandb_logger = WandbLogger(project="百度数据做开放关系BD_Knowledge Extractionnotebookb9f0237b84")
# /kaggle/input/reformerchinesemodel/epoch4step21209.ckpt
from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning.callbacks import ModelCheckpoint,LearningRateMonitor
# 自动停止
# https://pytorch-lightning.readthedocs.io/en/1.2.1/common/early_stopping.html
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
import torch.optim as optim

from optuna.visualization import plot_contour
from optuna.visualization import plot_edf
from optuna.visualization import plot_intermediate_values
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_slice




# 解决不显示图表问题
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
init_notebook_mode(connected=True)

In [4]:

# https://huggingface.co/uer/roberta-base-word-chinese-cluecorpussmall

# PATH="hfl/chinese-bert-wwm-ext"
# PATH="hfl/chinese-roberta-wwm-ext"
PATHV='uer/roberta-base-word-chinese-cluecorpussmall'
# PATH="bert-base-chinese"
# tokenizer = BertTokenizer.from_pretrained("uer/roberta-base-word-chinese-cluecorpussmall")
# tokenizer = AlbertTokenizer.from_pretrained("uer/roberta-base-word-chinese-cluecorpussmall")

# model = BertForMaskedLM.from_pretrained("uer/roberta-base-word-chinese-cluecorpussmall")

In [5]:
# tokenizer

In [6]:
dev=torch.load("../input/bdknowledgeextractionopenkg/dev.bin")
test=torch.load("../input/bdknowledgeextractionopenkg/test.bin")
train=torch.load("../input/bdknowledgeextractionopenkg/train.bin")

In [7]:
devdataset=TensorDataset(dev['input_ids'],dev['token_type_ids'],dev['attention_mask'],dev['labels'])
testdataset=TensorDataset(test['input_ids'],test['token_type_ids'],test['attention_mask'],test['labels'])
traindataset=TensorDataset(train['input_ids'],train['token_type_ids'],train['attention_mask'],train['labels'])

# 小数据截取测试用

In [8]:

# batch_size=80

# batch_size=72


# train_loader=DataLoader(traindataset,batch_size=batch_size, shuffle=True, )
# val_loader=DataLoader(devdataset,batch_size=batch_size, shuffle=False, )
# test_loader=DataLoader(testdataset,batch_size=batch_size, shuffle=False, )

In [9]:
class LitAutoLM(pl.LightningModule):
    """
    继承自bertlm模型
    做预测
    """
    def __init__(self, learning_rate=3e-4,T_max=500,optimizer_name="AdamW", **kwargs):
        super().__init__()
        self.hparams.config = BertConfig.from_pretrained("uer/roberta-small-word-chinese-cluecorpussmall")
        
        self.save_hyperparameters()
#         self.tokenizer = AlbertTokenizer.from_pretrained("uer/roberta-base-word-chinese-cluecorpussmall")
#         self.model = BertForMaskedLM.from_pretrained("uer/roberta-base-word-chinese-cluecorpussmall")
# https://huggingface.co/uer/roberta-tiny-word-chinese-cluecorpussmall
#         configuration = BertConfig.from_pretrained("uer/roberta-tiny-word-chinese-cluecorpussmall")

        self.tokenizer = AlbertTokenizer.from_pretrained("uer/roberta-tiny-word-chinese-cluecorpussmall")
        self.model = BertForMaskedLM.from_pretrained("uer/roberta-small-word-chinese-cluecorpussmall",config=self.hparams.config)

#         self.loss_fn = torch.nn.MSELoss()
#         tokenizer.save_vocabulary("./")
    def forward(self, input_ids,token_type_ids=None,attention_mask=None,labels=None):
        # in lightning, forward defines the prediction/inference actions
        outputs = self.model(input_ids=input_ids,token_type_ids=token_type_ids,attention_mask=attention_mask,labels=labels)
#         loss = outputs.loss
#         logits = outputs.logits
        return outputs
    def training_step(self, batch, batch_idx):
        # training_step defined the train loop.
        # It is independent of forward
        x, token_type_ids,attention_mask,y = batch
        outputs = self(x, token_type_ids,attention_mask,y)
        self.log('train_loss', outputs.loss)
        return outputs.loss
    def validation_step(self, batch, batch_idx):
        # training_step defined the train loop.
        # It is independent of forward
        x, token_type_ids,attention_mask,y = batch
        outputs = self(x, token_type_ids,attention_mask,y)
        self.log('val_loss', outputs.loss)
        self.log("hp_metric",outputs.loss) # 这个参数用于参数效果对比
        return outputs.loss

    def test_step(self, batch, batch_idx):
        # training_step defined the train loop.
        # It is independent of forward
        x, token_type_ids,attention_mask,y = batch
        outputs = self(x, token_type_ids,attention_mask,y)
        self.log('test_loss', outputs.loss)
        
        
        logits = outputs.logits
        for one,one2 in zip(logits.argmax(dim=-1).tolist(),y.tolist()):
            kgtext=[]
            kgtext2=[]
            for it,it2 in zip(one,one2):
    #             print(tokenizer.decode(it))
#                 if tokenizer.decode(it) in ["[SEP]","[PAD]"]:
#                     continue
                kgtext.append(tokenizer.decode(it))
                kgtext.append(tokenizer.decode(it2))
            print(kgtext)
            
            
        return outputs.loss
    def pred(self, text,ner1,ner2):
        # training_step defined the train loop.
        # It is independent of forward
        inputs = self.tokenizer([ner1+"和"+ner2+"关系为"+"".join(["[MASK]"]*10)],[text], return_tensors="pt", padding="max_length",max_length=128,truncation=True)
        outputs = self(**inputs)
        logits = outputs.logits
        kgtext=[]
        full=[]
        words=tokenizer.tokenize(text)
        for it in logits.argmax(dim=-1).tolist()[0]:
#             print(tokenizer.decode(it))
            full.append(tokenizer.decode(it))
            if tokenizer.decode(it) in ["[SEP]","[PAD]"]:
                continue
            
            kgtext.append(tokenizer.decode(it))
#         i="".join(kgtext).index('关系为')
#         return kgtext[i+3],kgtext
        return kgtext,full


    def pred2(self, text,ner1,p):
        # training_step defined the train loop.
        # It is independent of forward
        inputs = self.tokenizer([ner1+p+"为"+"".join(["[MASK]"]*10)],[text], return_tensors="pt", padding="max_length",max_length=128,truncation=True)
        outputs = self(**inputs)
        logits = outputs.logits
        kgtext=[]
        full=[]
        words=tokenizer.tokenize(text)
        for it in logits.argmax(dim=-1).tolist()[0]:
#             print(tokenizer.decode(it))
            full.append(tokenizer.decode(it))
            if tokenizer.decode(it) in ["[SEP]","[PAD]"]:
                continue
            
            kgtext.append(tokenizer.decode(it))
#         i="".join(kgtext).index('关系为')
#         return kgtext[i+3],kgtext
        return kgtext,full
#         x, token_type_ids,attention_mask = batch
#         outputs = self(x, token_type_ids,attention_mask)
#         self.log('test_loss', outputs.loss)
#         return outputs.loss
#     help(model.predict)
    def configure_optimizers(self):
            """优化器 # 类似于余弦，但其周期是变化的，初始周期为T_0,而后周期会✖️T_mult。每个周期学习率由大变小； https://www.notion.so/62e72678923f4e8aa04b73dc3eefaf71"""
    #         optimizer = torch.optim.AdamW(self.parameters(), lr=(self.learning_rate))

            #只优化部分
#             optimizer = torch.optim.AdamW(self.parameters(), lr=(self.hparams.learning_rate))
            if self.hparams.optimizer_name=="AdamW":
                optimizer = AdamW(self.parameters(), lr=(self.hparams.learning_rate))
            else:
                optimizer = getattr(optim, self.hparams.optimizer_name)(self.parameters(), lr=self.hparams.learning_rate)
    # https://pytorch.org/docs/stable/optim.html#torch.optim.Adadelta
#             optimizer = self.get_optimizer()
            #         使用自适应调整模型
            T_mult=2
            scheduler =torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer,T_0=self.hparams.T_max,T_mult=T_mult,eta_min=0 ,verbose=False)
    #         https://github.com/PyTorchLightning/pytorch-lightning/blob/6dc1078822c33fa4710618dc2f03945123edecec/pytorch_lightning/core/lightning.py#L1119

            lr_scheduler={
    #            'optimizer': optimizer,
               'scheduler': scheduler,
#                 'reduce_on_plateau': True, # For ReduceLROnPlateau scheduler
                'interval': 'step', #epoch/step
                'frequency': 1,
                'name':"lr_scheduler",
                'monitor': 'train_loss', #监听数据变化
                'strict': True,
            }
    #         return [optimizer], [lr_scheduler]
            return {"optimizer": optimizer, "lr_scheduler": lr_scheduler}

In [10]:
# help(DataLoader)

# /kaggle/input/reformerchinesemodel/epoch4step21209.ckpt
from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning.callbacks import ModelCheckpoint
# 自动停止
# https://pytorch-lightning.readthedocs.io/en/1.2.1/common/early_stopping.html
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks import LearningRateMonitor

# 引入修剪技术　https://pytorch-lightning.readthedocs.io/en/stable/advanced/pruning_quantization.html
from pytorch_lightning.callbacks import ModelPruning
import torch.nn.utils.prune as prune
# https://pytorch-lightning.readthedocs.io/en/latest/common/early_stopping.html
# 量化　降低内存　低精度　　https://pytorch-lightning.readthedocs.io/en/stable/advanced/pruning_quantization.html
from pytorch_lightning.callbacks import QuantizationAwareTraining

# 使用 DDP 时设置 find_unused_pa​​rameters=False
# 默认情况下，我们已启用查找未使用的参数为 True。这是针对过去出现的兼容性问题（有关更多信息，请参阅讨论）。默认情况下，这会影响性能，并且在大多数情况下可以禁用。
from pytorch_lightning.plugins import DDPPlugin

from pytorch_lightning.loggers import NeptuneLogger
from pytorch_lightning.loggers import WandbLogger
import wandb

# 使用小数据测试用

In [19]:
MAX=10000
# 小数据截取测试用
input_ids,token_type_ids,attention_mask,labels=devdataset[:int(MAX*0.2)]
devdataset=TensorDataset(input_ids,token_type_ids,attention_mask,labels)


input_ids,token_type_ids,attention_mask,labels=testdataset[:int(MAX*0.2)]
testdataset=TensorDataset(input_ids,token_type_ids,attention_mask,labels)


input_ids,token_type_ids,attention_mask,labels=traindataset[:MAX]
traindataset=TensorDataset(input_ids,token_type_ids,attention_mask,labels)

Problem finishing run
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/wandb/sdk/wandb_run.py", line 1502, in _atexit_cleanup
    self._on_finish()
  File "/opt/conda/lib/python3.7/site-packages/wandb/sdk/wandb_run.py", line 1667, in _on_finish
    self._poll_exit_response = self._wait_for_finish()
  File "/opt/conda/lib/python3.7/site-packages/wandb/sdk/wandb_run.py", line 1615, in _wait_for_finish
    poll_exit_resp = self._backend.interface.communicate_poll_exit()
  File "/opt/conda/lib/python3.7/site-packages/wandb/sdk/interface/interface.py", line 759, in communicate_poll_exit
    result = self._communicate(rec, timeout=timeout)
  File "/opt/conda/lib/python3.7/site-packages/wandb/sdk/interface/interface.py", line 533, in _communicate
    return self._communicate_async(rec, local=local).get(timeout=timeout)
  File "/opt/conda/lib/python3.7/site-packages/wandb/sdk/interface/interface.py", line 82, in get
    is_set = self._object_ready.wait(timeout)

In [20]:
  sweep_config = {
    "method": "bayes",   # Random search https://docs.wandb.ai/guides/sweeps/configuration#search-strategy
  "metric": {           # We want to maximize val_acc
#       "name": "val_acc",
#       "goal": "maximize"
      "name": "val_loss",
      "goal": "minimize",
#       "target": 0.99
  },
    
#   "method": "random",   # Random search https://docs.wandb.ai/guides/sweeps/configuration#search-strategy
  "parameters": {
        "batch_size":{
            "values": [12,24]
        },

        "optimizer_name": {
            # Choose from pre-defined values
            "values": ["AdamW"]
        },
      
         "learning_rate": {
            # log uniform distribution between exp(min) and exp(max)
            "distribution": "log_uniform",
            "min": -9.21,   # exp(-9.21) = 1e-4
            "max": -4.61    # exp(-4.61) = 1e-2
        },     
         "accumulate_grad_batches": {
            # Choose from pre-defined values
            "values": [1,2,3,4,5]
        },     
      

    }
}

In [16]:
sweep_id = wandb.sweep(sweep_config, project="optuna优化_百度数据做开放关系BD_KnowledgeExtractionnoteb")

Create sweep with ID: r81jxsys
Sweep URL: https://wandb.ai/terrychanorg/optuna%E4%BC%98%E5%8C%96_%E7%99%BE%E5%BA%A6%E6%95%B0%E6%8D%AE%E5%81%9A%E5%BC%80%E6%94%BE%E5%85%B3%E7%B3%BBBD_KnowledgeExtractionnoteb/sweeps/r81jxsys


In [ ]:
def sweep_iteration():
    # set up W&B logger
    wandb.init()    # required to have access to `wandb.config`
    wandb_logger = WandbLogger()
    train_loader=DataLoader(traindataset,batch_size=wandb.config.batch_size, shuffle=True, )
    val_loader=DataLoader(devdataset,batch_size=wandb.config.batch_size, shuffle=False, )
#     test_loader=DataLoader(testdataset,batch_size=wandb.config.batch_size, shuffle=False, )
    model=LitAutoLM(learning_rate=wandb.config.learning_rate,optimizer_name=wandb.config.optimizer_name)

    early_stop_callback = EarlyStopping(
       monitor='val_loss',
       min_delta=0.000,
       patience=20,
       verbose=True,
       mode='max'
    )
    lr_monitor = LearningRateMonitor(logging_interval='step')    
    
    
    trainer = pl.Trainer(
            gpus=1,
        #     min_epochs=1,
            precision=16,amp_level='O2',
#             checkpoint_callback=checkpoint_callback,
            callbacks=[],
    #         resume_from_checkpoint="/kaggle/input/openkgmodelml/wandb/run-20210610_054617-3cu3oiy2/files/百度数据做开放关系BD_Knowledge\ Extractionnotebookb9f0237b84/3cu3oiy2/checkpoints/chinese-out.ckpt",
            auto_select_gpus=True,
#             callbacks=[lr_monitor,early_stop_callback],
            deterministic=True,
    #         auto_scale_batch_size='binsearch',
    #         auto_lr_find=True,
    #         max_epochs=wandb.config.epochs,
            max_epochs=5,
            logger=wandb_logger,
    #         accumulate_grad_batches=wandb.config.accumulate_grad_batches

            accumulate_grad_batches=wandb.config.accumulate_grad_batches)
    # train
    trainer.fit(model,train_dataloader=train_loader,val_dataloaders=val_loader)

    
count = 50 # number of runs to execute
wandb.agent(sweep_id, function=sweep_iteration,count =count)
# while not sweep.done():
#     sweep.print_status()
#     sweep.step()
#     time.sleep(5)

Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [ ]:
#     train_loader=DataLoader(traindataset,batch_size=batch_size, shuffle=True, )
#     val_loader=DataLoader(devdataset,batch_size=batch_size, shuffle=False, )
#     test_loader=DataLoader(testdataset,batch_size=batch_size, shuffle=False, )
#     model=LitAutoLM(learning_rate=learning_rate,optimizer_name=optimizer_name)
    
#     trainer = pl.Trainer(
#             gpus=1,
#         #     min_epochs=1,
#             precision=16,amp_level='O2',
# #             checkpoint_callback=checkpoint_callback,
#             callbacks=[PyTorchLightningPruningCallback(trial, monitor="val_loss")],
#     #         resume_from_checkpoint="/kaggle/input/openkgmodelml/wandb/run-20210610_054617-3cu3oiy2/files/百度数据做开放关系BD_Knowledge\ Extractionnotebookb9f0237b84/3cu3oiy2/checkpoints/chinese-out.ckpt",
#             auto_select_gpus=True,
# #             callbacks=[lr_monitor,early_stop_callback],
#             deterministic=True,
#     #         auto_scale_batch_size='binsearch',
#     #         auto_lr_find=True,
#     #         max_epochs=wandb.config.epochs,
#             max_epochs=1,
#     #         logger=wandb_logger,
#     #         accumulate_grad_batches=wandb.config.accumulate_grad_batches

#             accumulate_grad_batches=2)


# optuna 优化

https://optuna.readthedocs.io/zh_CN/latest/reference/generated/optuna.trial.Trial.html?highlight=suggest_float#optuna.trial.Trial.suggest_loguniform

In [ ]:
# import logging,sys
# def objective(trial: optuna.trial.Trial) -> float:
#     # wandb_logger = WandbLogger(name='nerchaijie拆解',project="colab_optuna 使用pytorch_lightning超参优化 .ipynb")
#     # wandb.init()
#     # name=str(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))
#     # name=str(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))+str(num)
#     # num=num+1
#     # wandb_logger = WandbLogger(project="colab_optuna 使用pytorch_lightning超参优化 .ipynb")
#     # wandb_logger = WandbLogger()
#     # We optimize the number of layers, hidden units in each layer and dropouts.
# #     n_layers = trial.suggest_int("n_layers", 1, 3)
#     learning_rate = trial.suggest_loguniform("learning_rate", 1e-5, 1e-3)
# #     max_epochs = trial.suggest_int("max_epochs", 1, 1)
#     batch_size = trial.suggest_categorical("batch_size", [12])
# # "MomentumSGD",
# #     optimizer_name = trial.suggest_categorical("optimizer", ["Adam","AdamW", "RMSprop", "SGD"])
#     optimizer_name = trial.suggest_categorical("optimizer", ["AdamW"])
#     accumulate_grad_batches=trial.suggest_int("accumulate_grad_batches", 1, 5)
# #     batch_size=72


#     train_loader=DataLoader(traindataset,batch_size=batch_size, shuffle=True, )
#     val_loader=DataLoader(devdataset,batch_size=batch_size, shuffle=False, )
#     test_loader=DataLoader(testdataset,batch_size=batch_size, shuffle=False, )
#     model=LitAutoLM(learning_rate=learning_rate,optimizer_name=optimizer_name)
    
#     trainer = pl.Trainer(
#             gpus=1,
#         #     min_epochs=1,
#             precision=16,amp_level='O2',
# #             checkpoint_callback=checkpoint_callback,
#             callbacks=[PyTorchLightningPruningCallback(trial, monitor="val_loss")],
#     #         resume_from_checkpoint="/kaggle/input/openkgmodelml/wandb/run-20210610_054617-3cu3oiy2/files/百度数据做开放关系BD_Knowledge\ Extractionnotebookb9f0237b84/3cu3oiy2/checkpoints/chinese-out.ckpt",
#             auto_select_gpus=True,
# #             callbacks=[lr_monitor,early_stop_callback],
#             deterministic=True,
#     #         auto_scale_batch_size='binsearch',
#     #         auto_lr_find=True,
#     #         max_epochs=wandb.config.epochs,
#             max_epochs=1,
#     #         logger=wandb_logger,
#     #         accumulate_grad_batches=wandb.config.accumulate_grad_batches

#             accumulate_grad_batches=2)

    
#     hyperparameters = dict(learning_rate=learning_rate,batch_size=batch_size,optimizer_name=optimizer_name)
#     print("hyperparameters",hyperparameters)
#     trainer.logger.log_hyperparams(hyperparameters)
    
#     trainer.fit(model, train_loader,val_loader)
#     # still doesn't work
#     val_loss=trainer.callback_metrics["val_loss"].item()
#     del model
# #     del trainer
#     torch.cuda.empty_cache()
#     return val_loss



# study = optuna.create_study(direction="minimize", pruner=optuna.pruners.MedianPruner())
# study.optimize(objective, n_trials=100, timeout=600)



# print("Number of finished trials: {}".format(len(study.trials)))

# print("Best trial:")
# trial = study.best_trial

# print("  Value: {}".format(trial.value))

# print("  Params: ")
# for key, value in trial.params.items():
#     print("    {}: {}".format(key, value))

In [ ]:
# print(f"Sampler is {study.sampler.__class__.__name__}")

In [ ]:
# print("Best trial:")
# trial = study.best_trial
# print("  Params: ")
# for key, value in trial.params.items():
#     print("    {}: {}".format(key, value))


# 可视化

https://optuna.readthedocs.io/zh_CN/latest/tutorial/10_key_features/005_visualization.html#sphx-glr-tutorial-10-key-features-005-visualization-py

In [ ]:
# plot_optimization_history(study)

In [ ]:
# plot_parallel_coordinate(study)

# 恢复大数据训练


In [ ]:
devdataset=TensorDataset(dev['input_ids'],dev['token_type_ids'],dev['attention_mask'],dev['labels'])
testdataset=TensorDataset(test['input_ids'],test['token_type_ids'],test['attention_mask'],test['labels'])
traindataset=TensorDataset(train['input_ids'],train['token_type_ids'],train['attention_mask'],train['labels'])



MAX=1000000000
# 小数据截取测试用
input_ids,token_type_ids,attention_mask,labels=devdataset[:int(MAX*0.2)]
devdataset=TensorDataset(input_ids,token_type_ids,attention_mask,labels)


input_ids,token_type_ids,attention_mask,labels=testdataset[:int(MAX*0.2)]
testdataset=TensorDataset(input_ids,token_type_ids,attention_mask,labels)


input_ids,token_type_ids,attention_mask,labels=traindataset[:MAX]
traindataset=TensorDataset(input_ids,token_type_ids,attention_mask,labels)

In [ ]:
print("Best trial:")
trial = study.best_trial
print("  Params: ")

best_params={}
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))
    best_params[key]=value
    



train_loader=DataLoader(traindataset,batch_size=best_params["batch_size"], shuffle=True, )
val_loader=DataLoader(devdataset,batch_size=best_params["batch_size"], shuffle=False, )
test_loader=DataLoader(testdataset,batch_size=best_params["batch_size"], shuffle=False, )
model=LitAutoLM(learning_rate=best_params["learning_rate"],optimizer_name=best_params["optimizer"])

wandb_logger = WandbLogger(project="optuna优化_百度数据做开放关系BD_KnowledgeExtractionnoteb")

seed_everything(42)

early_stop_callback = EarlyStopping(
   monitor='val_loss',
   min_delta=0.0000,
   patience=15,
   verbose=True,
   mode='min',
#   check_finite=True,
#  check_on_train_epoch_end=True,
#     divergence_threshold=0.1
    
)

!mkdir /kaggle/working/checkpoints
checkpoint_callback = ModelCheckpoint(
#     filename='/kaggle/working/{epoch}-{val_loss:.2f}',
    dirpath="/kaggle/working/checkpoints",
#     filename='{epoch:02d}-{step}-{val_loss:.2f}',
    filename='out',
#     save_last=True,
    verbose=True,
    monitor='val_loss',
#     every_n_train_steps=2,
    mode='min',
#     best_model_path='best'
    save_top_k=2
)
lr_monitor = LearningRateMonitor(logging_interval='step')







trainer = pl.Trainer(
        gpus=1,
    #     min_epochs=1,
        precision=16,amp_level='O2',
        val_check_interval=0.25, #这里增加检查验证的频率
        limit_val_batches=0.25, # 限制验证的数目 降低每次验证的批次大小
        checkpoint_callback=True,
#             checkpoint_callback=checkpoint_callback,
#         callbacks=[PyTorchLightningPruningCallback(trial, monitor="val_loss")],
        resume_from_checkpoint="../input/optuna-bd-knowledgeextractionnoteb/checkpoints/epoch=6-step=28455.ckpt",
        auto_select_gpus=True,
 
        callbacks=[lr_monitor,early_stop_callback,
#                pruning,
               checkpoint_callback
        #                        QuantizationAwareTraining()
              ],
 
#             callbacks=[lr_monitor,early_stop_callback],
        deterministic=True,
#         auto_scale_batch_size='binsearch',
#         auto_lr_find=True,
#         max_epochs=wandb.config.epochs,
        max_epochs=10,
        logger=wandb_logger,
#         accumulate_grad_batches=wandb.config.accumulate_grad_batches
#         terminate_on_nan=True, # 出现nan则停止
        accumulate_grad_batches=2)



# trainer.fit(model, train_loader,val_loader)


In [ ]:


# lr_monitor = LearningRateMonitor(logging_interval='step')
# early_stop_callback = EarlyStopping(
#    monitor='train_loss',
#    min_delta=0.00,
#    patience=10,
#    verbose=True,
#    mode='min'
# )
# checkpoint_callback = ModelCheckpoint(
#     filename='chinese-out',
#     verbose=True,
#     monitor='train_loss',
#     mode='min',
# #     best_model_path='best'
#     save_top_k=2
# )



# model=LitAutoLM()
# trainer = pl.Trainer(
#         gpus=1,
#     #     min_epochs=1,
#         precision=16,amp_level='O2',
#         checkpoint_callback=checkpoint_callback,
# #         resume_from_checkpoint="/kaggle/input/openkgmodelml/wandb/run-20210610_054617-3cu3oiy2/files/百度数据做开放关系BD_Knowledge\ Extractionnotebookb9f0237b84/3cu3oiy2/checkpoints/chinese-out.ckpt",
#         auto_select_gpus=True,
#         callbacks=[lr_monitor,early_stop_callback],
#         deterministic=True,
# #         auto_scale_batch_size='binsearch',
# #         auto_lr_find=True,
# #         max_epochs=wandb.config.epochs,
#         max_epochs=5,
# #         logger=wandb_logger,
# #         accumulate_grad_batches=wandb.config.accumulate_grad_batches

#         accumulate_grad_batches=2)

# trainer.fit(model, train_loader,val_loader)

In [ ]:
# from transformers import pipeline
# # https://huggingface.co/uer/roberta-base-word-chinese-cluecorpussmall
# # unmasker = pipeline('fill-mask', model='uer/roberta-medium-word-chinese-cluecorpussmall')
# unmasker = pipeline('fill-mask', model='uer/roberta-base-word-chinese-cluecorpussmall')
# unmasker("如何演好自己的角色，请读《演员自我修养》《喜剧之王》周星驰崛起于穷困潦倒之中的独门秘笈 [SEP] 喜剧之王和周星驰关系为 [MASK]")

In [ ]:
#trainer.test(test_dataloaders=test_loader)

In [ ]:
# dir(model)

In [ ]:

torch.save(model.state_dict(), "model.bin")

# model = TheModelClass(*args, **kwargs)
# model.load_state_dict(torch.load(PATH))
# model.eval()

In [ ]:
model=LitAutoLM()
model.load_state_dict(torch.load("model.bin"))
# model.load_state_dict(torch.load("../input/openkgmodelml/model.bin"))


model.eval()
pass

In [ ]:
model.eval()
out=model.pred("硫磺溪是一条位于美国加利福尼亚州橙县的溪流，亦是亚里索溪的支流。长约7.7公里，流域面积约16平方公里。","橙县","美国加利福尼亚州")
print(out)

index="".join(out[0]).index('关系为')

predicate="".join(out[0])[index+3:]


print(predicate)


out=model.pred2("硫磺溪是一条位于美国加利福尼亚州橙县的溪流，亦是亚里索溪的支流。长约7.7公里，流域面积约16平方公里。","硫磺溪","位于")
print(out)

# index="".join(out[0]).index('关系为')

# predicate="".join(out[0])[index+3:]


# print(predicate)

# 测试模型效果

In [ ]:
tokenizer = AlbertTokenizer.from_pretrained("uer/roberta-base-word-chinese-cluecorpussmall")
# model = BertForMaskedLM.from_pretrained("uer/roberta-base-word-chinese-cluecorpussmall")

inputs = tokenizer(["如何演好自己的角色，请读《演员自我修养》《喜剧之王》周星驰崛起于穷困潦倒之中的独门秘笈 "],[" 喜剧之王和周星驰关系为 [MASK] [MASK] "], return_tensors="pt", padding="max_length",max_length=128,truncation=True)
labels = tokenizer(["如何演好自己的角色，请读《演员自我修养》《喜剧之王》周星驰崛起于穷困潦倒之中的独门秘笈 "],[" 喜剧之王和周星驰关系为主演"], return_tensors="pt", padding="max_length",max_length=128,truncation=True)["input_ids"]

outputs = model(**inputs, labels=labels)
loss = outputs.loss
logits = outputs.logits

print(inputs)

In [ ]:
loss

In [ ]:
for it in logits.argmax(dim=-1).tolist()[0]:
    print(tokenizer.decode(it))